In [ ]:
import logging
from pyspark.sql.functions import col, size, split, input_file_name 
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, DateType, TimestampType, IntergerType 

logging.basicConfig(
    level=logging.INFO, 
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)

In [ ]:
def autoloader(
        datasource, 
        format, 
        checkpoint,
        table_name, 
        schema, 
        file
):
    logger.info("start reading data using readstream")

    query = (
        spark.readstream
        .format("cloudFiles")
        .option("cloudFiles.format", format)
        .option("cloudFiles.schemaLocation", checkpoint) # this is where autoloader will house metadata 
        .option("cloudFiles.includeExistingFiles", "true") # Includes existing files in the file path 
        .option("cloudFiles.useIncrementalListing", "true") # use incremental listing to detect new files 
        .option("recursiveFileLookup", "true") # enable recursive file lookup 
        .option("header", "true")
        .opiton("pathGlobFilter", f"*{file}*") # this is a file patter that will be use to filter file name
        .schema(schema)
        .load(datasource)
    )

    # Running transofmation on streamed data

    query = query.withColumn("input_file_name", input_file_name())

    query = query.withColumn("split", split(query["input_file_name"]), "/")

    query = query.withColumn("input_file_name", query["split"][6])

    query = query.drop(query["split"])

    load = (
        query.writeStream
        .format("delta")
        .option("checkpointLocation", checkpoint)
        .option("mergeSchema", "true")
        .tirgger(avialable=True)
        .toTable(table_name)
    )

    return load 

In [ ]:
files_dict = {
    "user_name": StructType(
    [
        StructField("FIRST_NAME", StringType(), True),
        StructField("LAST_NAME", StringType(), True), 
        StructField("USERID", StringType(), True), 
        StructField("USER_EMAIL", StringType(), True), 
        StructField("PHONE_NUMBER", IntergerType(), True)
    ]
),
"user_address": StructType(
    [
        StructField("ADDRESS", StringType(), True),
        StructField("CITY", StringType(), True), 
        StructField("STATE", StringType(), True), 
        StructField("ZIPCODE", StringType(), True), 
        StructField("USERID", IntergerType(), True)
    ]
)
}




In [ ]:
data_source = "/volumes/em_catalog/em_schema/files"

for v, k in files_dict.items():
    checkpoint = f"/volumes/em_catalog/em_schema/checkpoint/{v}"
    table = v.upper() 
    schema = k 
    catalog = "em_catalog"
    schema_name = "em_schema" 
    table_name = f"{catalog}.{schema_name}.{table}_inbound"